In [2]:
from ast import *
def _sub_subtrees_for_vars(ast, ast_mappings):
    for node in walk(ast):
        if isinstance(node, Name) and ast_mappings.has_key(unparse(ast)):
            node = ast_mappings[unparse(ast)]
    return ast
def strip_parse(expr):
    return parse(expr)
def sub_for_func(expr, func_name, func_vars, func_expr):
    ast = strip_parse(expr)
    func_name_ast = strip_parse(func_name)
    nodes = [node for node in walk(func_name_ast.body[0])]
    print(nodes)
    if not isinstance(nodes[1], Name):
        raise ValueError('Function name is not a simple name.')
    
    func_name = func_name_ast.body[0].value.id
    print(func_name)
    func_expr_ast = strip_parse(func_expr)
    # We can strip_parse  the '*', so we special case it here.
    if func_vars == '*':
        if not hasattr(func_expr_ast, 'nodes'):
            raise ValueError("Top-level function in %s does not appear to "
                             "accept variable number of arguments. (It has no "
                             "'nodes' attribute.)" % func_expr)

        func_var_names = '*'
    else:
        func_vars_ast = [strip_parse(var).body[0].value for var in func_vars]
        for var_ast in func_vars_ast:
            if not isinstance(var_ast, Name):
                raise ValueError('Function variable is not a simple name.')
        func_var_names = [getattr(var_ast, 'id') for var_ast in func_vars_ast]
        print(func_var_names)

    ast = _sub_for_func_ast(ast, func_name, func_var_names, func_expr_ast)
#     simple = Simplify._simplify_ast(ast)
    return ast


def _sub_for_func_ast(ast, func_name, func_vars, func_expr_ast):

    print(dump(ast), func_name, func_vars, dump(func_expr_ast))
    for node in walk(ast):
    
        if isinstance(node, Call) and unparse(node.value.id) == func_name\
           and func_vars == '*':
            working_ast = copy.deepcopy(func_expr_ast)
            new_args = [_sub_for_func_ast(arg_ast, func_name, func_vars, 
                                          func_expr_ast) for arg_ast in node.value.args]
            # This subs out the arguments of the original function.
            working_ast.node.value.args = new_args
            return working_ast
        if isinstance(ast, Call) and unparse(node.value.id) == func_name\
           and len(node.value.args) == len(func_vars):
            # If our ast is the function we're looking for, we take the ast
            #  for the function expression, substitute for its arguments, and
            #  return
            working_ast = copy.deepcopy(func_expr_ast)
            mapping = {}
            for var_name, arg_ast in zip(func_vars, node.value.args):
                subbed_arg_ast = _sub_for_func_ast(arg_ast, func_name, func_vars, 
                                                   func_expr_ast)
                mapping[var_name] = subbed_arg_ast
            _sub_subtrees_for_vars(working_ast, mapping)
            return working_ast
    return ast
    

In [3]:
def test_sub_for_func():
    cases = [('f(x)', 'f', 'y', 'y+1',
              'x+1'),
             ]

    for expr, func_name, func_vars, func_expr, answer in cases:
        print
        subbed = sub_for_func(expr, func_name, func_vars,
                                           func_expr)
        assert eval(answer) == eval(subbed)

In [4]:
test_sub_for_func()

[<ast.Expr object at 0x7fc70d858490>, <ast.Name object at 0x7fc70d858040>, <ast.Load object at 0x7fc7113ccb50>]
f
['y']
Module(body=[Expr(value=Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))], type_ignores=[]) f ['y'] Module(body=[Expr(value=BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=1)))], type_ignores=[])


AttributeError: 'Call' object has no attribute 'value'

In [5]:
import ast
flags = ast.PyCF_ONLY_AST 
tree = compile("a+b", filename='<unknown>', mode='exec',
    flags=flags, dont_inherit=True)

In [1]:
import sys
sys.executable

'/usr/bin/python3.9'

In [52]:
def test__collect_num_denom():
        cases = [
                 (strip_parse('1 + 3/2 + 4'), (['1'], ['2'])),
                 ]
        for ast, (nums, denoms) in cases: 
            n, d = [], []
            print(dump(ast))
            _collect_num_denom(ast, n, d)
            n = [unparse(term) for term in n]
            d = [unparse(term) for term in d]
#             print("numerator", nums)
#             print("denominator", denoms)
#             print(n)
#             print(d)
#             assert set(nums) == set(n)
#             assert set(denoms) == set(d)

In [56]:
test__collect_num_denom()

Module(body=[Expr(value=BinOp(left=BinOp(left=Constant(value=1), op=Add(), right=BinOp(left=Constant(value=3), op=Div(), right=Constant(value=2))), op=Add(), right=Constant(value=4)))], type_ignores=[])
numerator ['1']
denominator ['2']
[]
[]


In [60]:
class PowForDoubleStar(NodeVisitor):
    nums = nums
    denoms = denoms
    def visit_BinOp(self, node):
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        print(node)
        print(node)
        if not (isinstance(node.op, Mult) or isinstance(node.op, Div)):
            self.nums.append(node)
           
        if isinstance(node.left, Div) or isinstance(node.left, Mult):
            _collect_num_denom(node.left, nums, denoms)
        else:
            self.nums.append(node.left)
        if isinstance(node.right, Div) or isinstance(node.right, Mult):
            if isinstance(node, Mult):
                _collect_num_denom(node.right, nums, denoms)
            elif isinstance(node, Div):
                _collect_num_denom(node.right, denoms, nums)
        else:
            if isinstance(node, Mult):
                self.nums.append(node.right)
            elif isinstance(node, Div):
                denoms.append(node.right)
        self.generic_visit(node)
        

      
    
def collect_num_denom(tree, nums, denoms):
    """
    Append to nums and denoms, respectively, the nodes in the numerator and 
    denominator of an AST.
    """
    PowForDoubleStar().visit(tree, nums, denoms)

NameError: name 'nums' is not defined

In [154]:
x = "1/2"
def strip_parse(x):
    return parse(x).body[0].value
nums = []
denoms = []
class PowForDoubleStar(NodeVisitor):
    def visit_BinOp(self, node):
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        print(node.op)
        if not (isinstance(node.op, Mult) or isinstance(node.op, Div)):
            print("here", node)
           
#         if isinstance(node.left, Div) or isinstance(node.left, Mult):
#             print("entered here")
#         else:
#             nums.append(node.left)
#         if isinstance(node.right, Div) or isinstance(node.right, Mult):
#             if isinstance(node, Mult):
#                 _collect_num_denom(node.right, nums, denoms)
#             elif isinstance(node, Div):
#                 _collect_num_denom(node.right, denoms, nums)
#         else:
#             if isinstance(node, Mult):
#                 nums.append(node.right)
#             elif isinstance(node, Div):
#                 denoms.append(node.right)
        self.generic_visit(node)
tree = strip_parse(x)
print(dump(tree))
PowForDoubleStar().visit(tree)
print(nums)
print(denoms)

BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2))
[]
[]


In [149]:
x = "(1/2)/3"
tree = strip_parse(x)
print(dump(tree))
# print(tree.body[0].value)
nums = []
denoms = []
# print("children", tree.iter_child_nodes)
_collect_num_denom(tree.body[0].value, nums, denoms)
print("nums", nums)

print("denoms", denoms)

Module(body=[Expr(value=BinOp(left=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)), op=Div(), right=Constant(value=3)))], type_ignores=[])
nums [1]
denoms [2, 3]


In [144]:
x = "1/2"
tree = strip_parse(x)
print(dump(tree))
print(tree.body[0].value)
print(tree.body[0].value.left)
print(tree.body[0].value.right)

Module(body=[Expr(value=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)))], type_ignores=[])


In [199]:
def test__collect_num_denom():
        cases = [(strip_parse('1'), (['1'], [])),
                 (strip_parse('1/2'), (['1'], ['2'])),
                 (strip_parse('1/2*3'), (['1', '3'], ['2'])),
#                  (strip_parse('1/(2*3)'), (['1'], ['2', '3'])),
#                  (strip_parse('1/(2/3)'), (['1', '3'], ['2']))
                 ]
        for ast, (nums, denoms) in cases: 
            n, d = [], []
            print(dump(ast))
            _collect_num_denom(ast, n, d)
           
            n = [term for term in n]
            d = [term for term in d]
            nums = [int(term) for term in nums]
            denoms = [int(term) for term in denoms]
           
            print(n, nums)
            print(d, denoms)
            print("-------------")
            assert set(nums) == set(n)
            assert set(denoms) == set(d)
test__collect_num_denom()

Constant(value=1)
[1] [1]
[] []
-------------
BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2))
[1] [1]
[2] [2]
-------------
BinOp(left=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)), op=Mult(), right=Constant(value=3))


AttributeError: 'BinOp' object has no attribute 'value'

In [200]:
def _collect_num_denom(ast, nums, denoms):
    """
    Append to nums and denoms, respectively, the nodes in the numerator and 
    denominator of an AST.
    """
    if not isinstance(ast, BinOp):
        nums.append(ast.value)
        return
    if (isinstance(ast.op, Div) or isinstance(ast.op, Mult)):
        if isinstance(ast.left, BinOp):
            _collect_num_denom(ast.left, nums, denoms)
        else:
            nums.append(ast.left.value)
            print("here", nums)
            
        if isinstance(ast.right, BinOp):
            if isinstance(ast.op, Div):
                _collect_num_denom(ast.right, denoms, nums)
            elif isinstance(ast.op, Mult):
                _collect_num_denom(ast.left, nums, denoms)
        else:
            if isinstance(ast.op, Div):
                denoms.append(ast.right.value)
            elif isinstance(ast.op, Mult):
                nums.append(ast.right.value)

In [142]:
def ast2str(ast, outer = _FARTHEST_OUT, adjust = 0):
    """
    Return the string representation of an AST.

    outer: The AST's 'parent' node, used to determine whether or not to 
        enclose the result in parentheses. The default of _FARTHEST_OUT will
        never enclose the result in parentheses.

    adjust: A numerical value to adjust the priority of this ast for
        particular cases. For example, the denominator of a '/' needs 
        parentheses in more cases than does the numerator.
    """
    if isinstance(ast, Name):
        out = ast.name
    elif isinstance(ast, Const):
        out = str(ast.value)
    elif isinstance(ast, Add):
        out = '%s + %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Sub):
        out = '%s - %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Mul):
        out = '%s*%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Div):
        # The adjust ensures proper parentheses for x/(y*z)
        out = '%s/%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Power):
        # The adjust ensures proper parentheses for (x**y)**z
        out = '%s**%s' % (ast2str(ast.left, ast, adjust = TINY),
                          ast2str(ast.right, ast))
    elif isinstance(ast, UnarySub):
        out = '-%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, UnaryAdd):
        out = '+%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, CallFunc):
        args = [ast2str(arg) for arg in ast.args]
        out = '%s(%s)' % (ast2str(ast.node), ', '.join(args))
    elif isinstance(ast, Subscript):
        subs = [ast2str(sub) for sub in ast.subs]
        out = '%s[%s]' % (ast2str(ast.expr), ', '.join(subs))
    elif isinstance(ast, Slice):
        out = '%s[%s:%s]' % (ast2str(ast.expr), ast2str(ast.lower), 
                             ast2str(ast.upper))
    elif isinstance(ast, Sliceobj):
        nodes = [ast2str(node) for node in ast.nodes]
        out = ':'.join(nodes)
    elif isinstance(ast, Compare):
        expr = ast2str(ast.expr, ast, adjust=6+TINY)
        out_l = [expr]
        for op, val in ast.ops:
            out_l.append(op)
            out_l.append(ast2str(val, ast, adjust=6+TINY))
        out = ' '.join(out_l)
    elif isinstance(ast, And):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' and '.join(nodes)
    elif isinstance(ast, Or):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' or '.join(nodes)
    elif isinstance(ast, Not):
        out = 'not %s' % ast2str(ast.expr, ast, adjust=TINY)

    # Ensure parentheses by checking the _OP_ORDER of the outer and inner ASTs
    if _need_parens(outer, ast, adjust):
        return out
    else:
        return '(%s)' % out

In [184]:
dump(strip_parse('0'))


'Constant(value=0)'

[1] []
[1] [2]
[1, 3] [2]
[1] [2, 3]
[1, 3] [2]
[1, 4] [2, 3]
